In [1]:
import os
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate

from dotenv import dotenv_values
config = dotenv_values(".env")
print(config['IP_OLLAMA'])

https://ng92lp3ynw3p3o-11434.proxy.runpod.net


In [2]:
## LOAD VECTOR DB YANG SUDAH DIBUAT SEBELUMNYA
def load_models(data_path):
    if os.path.exists(data_path):
        print("------[LOG]------\n", data_path, "\n-----------------")
        vector_store = FAISS.load_local(
            data_path,
            OllamaEmbeddings(model="bge-m3", base_url=config['IP_OLLAMA']),
            allow_dangerous_deserialization=True
        )
        return vector_store
    else:
        raise FileNotFoundError(f"Data path does not exist: {data_path}")

In [3]:
retrievers = load_models("/Users/birutekno/Documents/Project/soca-ml-app/materi-gdg/vectoring/embeddings")

------[LOG]------
 /Users/birutekno/Documents/Project/soca-ml-app/materi-gdg/vectoring/embeddings 
-----------------


In [4]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import StuffDocumentsChain, RetrievalQAWithSourcesChain
from langchain_ollama import ChatOllama

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""kamu adalah bot pintar menggunakan gemma2, jawab pertanyaan user berdasarkan context dibawah. selalu jawab pertanyaan user menggunakan bahasa indonesia. jika kamu tidak tahu jawaban nnya sebutkan saja "maaf saya tidak tahu", jangan coba untuk memberikan jawaban yang tidak relevan.
    Context: {context}\n
    Question: {question}\n
    Answer:"""
)

ollama_model = "gemma2:9b"
api_url = config['IP_OLLAMA']

llm = ChatOllama(
    model=ollama_model,
    base_url=api_url
)

llm_chain = LLMChain(llm=llm, prompt=prompt_template)

class VerboseStuffDocumentsChain(StuffDocumentsChain):
    def combine_docs(self, docs, **kwargs):
        for doc in docs:
            print(f"[DEBUG] Chunk to LLM: {doc.page_content}")
        return super().combine_docs(docs, **kwargs)

stuff_chain = VerboseStuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
)

retriever = retrievers.as_retriever(search_type="similarity", search_kwargs={"k": 3})
qa_chain = RetrievalQAWithSourcesChain(
    combine_documents_chain=stuff_chain,
    retriever=retriever,
    return_source_documents=True,
    verbose=True
)

result = qa_chain({"question": "menjabat jadi mentri apakah agus hari murti yudhoyono?"})

source_list = [str(doc.metadata['source']).split('/')[-1] for doc in result['source_documents'] if str(doc.metadata['source']).split('.')[-1] not in ['jpg', 'png']]
source_list = list(set(source_list))

print(result['answer'])
print(source_list)


/var/folders/m4/_5r061fs01dfpsm_ydkg8_f00000gn/T/ipykernel_45702/3005256803.py:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt_template)
/var/folders/m4/_5r061fs01dfpsm_ydkg8_f00000gn/T/ipykernel_45702/3005256803.py:43: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"question": "menjabat jadi mentri apakah agus hari murti yudhoyono?"})




> Entering new RetrievalQAWithSourcesChain chain...
[DEBUG] Chunk to LLM: 16.
Sri Mulyani Indrawati, Menteri Keuangan;
17.
Abdul Mu’ti, Menteri Pendidikan Dasar dan Menengah;
18.
Satryo Soemantri Brodjonegoro, Menteri Pendidikan
Tinggi, Sains, dan Teknologi;
19.
[DEBUG] Chunk to LLM: 3.
Airlangga
Hartarto,
Menteri
Koordinator
Bidang
Perekonomian;
4.
Pratikno, Menteri Koordinator Bidang Pembangunan
Manusia dan Kebudayaan;
5.
Agus
Harimurti
Yudhoyono,
Menteri
Koordinator
[DEBUG] Chunk to LLM: 5.
Agus
Harimurti
Yudhoyono,
Menteri
Koordinator
Bidang Infrastruktur dan Pembangunan Kewilayahan:
6.
Abdul Muhaimin Iskandar, Menteri Koordinator Bidang
Pemberdayaan Masyarakat;
7.


ResponseError: model "gemma2:9b" not found, try pulling it first

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.chains.qa_with_sources.retrieval import RetrievalQAWithSourcesChain
from langchain_ollama import ChatOllama
from langchain.schema.runnable import RunnableMap, RunnableLambda

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""kamu adalah bot pintar menggunakan gemma2, jawab pertanyaan user berdasarkan context dibawah. 
    selalu jawab pertanyaan user menggunakan bahasa indonesia. jika kamu tidak tahu jawabannya, sebutkan saja 
    "maaf saya tidak tahu", jangan coba untuk memberikan jawaban yang tidak relevan.

    Context: {context}\n
    Question: {question}\n
    Answer:"""
)

ollama_model = "gen-inst-1:latest"
api_url = config["IP_OLLAMA"]

llm = ChatOllama(
    model=ollama_model,
    base_url=api_url
)

retriever = retrievers.as_retriever(
    search_type="similarity", search_kwargs={"k": 3}
)
def debug_documents(inputs):
    """
    Debug and print the content of each document fetched by the retriever.
    """
    docs = inputs.get("documents", [])
    print("[DEBUG] Documents fetched by retriever:")
    if not docs:
        print("[DEBUG] No documents found by retriever.")
        return inputs
    
    for idx, doc in enumerate(docs, 1):
        print(f"\n[DEBUG] Chunk {idx} (Source: {doc.metadata.get('source', 'unknown')}):")
        print(doc.page_content[:500])  # Print first 500 characters for readability
    return inputs

# Updated pipeline to properly chain steps
pipeline = RunnableMap({
    "retriever": RunnableLambda(lambda inputs: {"documents": retriever.get_relevant_documents(inputs["question"])}),
    "debugger": RunnableLambda(lambda inputs: debug_documents(inputs)),
    "qa_chain": RetrievalQAWithSourcesChain.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt_template, "document_variable_name": "context"},
        return_source_documents=True,
        verbose=True,
    ),
})

# Invoke the pipeline with the question
result = pipeline.invoke({
    "question": "jelaskan spesifikasi telin1 singapore data centre telin"
})

# Extract answer and source documents
answer = result["qa_chain"]["answer"]
source_list = [
    str(doc.metadata["source"]).split("/")[-1]
    for doc in result["qa_chain"]["source_documents"]
    if str(doc.metadata["source"]).split(".")[-1] not in ["jpg", "png"]
]
source_list = list(set(source_list))

# Output the final answer and source documents
print("\nFinal Answer:")
print(answer)

print("\nSource Documents:")
print(source_list)



> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.
Sri Mulyani Indrawati adalah menteri keuangan dalam Kabinet Merah Putih periode 2024-2029.
['nama_mentri.pdf']


In [6]:
result = pipeline.invoke({
    "question": "siapa presiden indonesia 2024  ?"
})

answer = result["qa_chain"]["answer"]
source_list = [
    str(doc.metadata["source"]).split("/")[-1]
    for doc in result["qa_chain"]["source_documents"]
    if str(doc.metadata["source"]).split(".")[-1] not in ["jpg", "png"]
]
source_list = list(set(source_list))

print(answer)
print(source_list)




> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.
Presiden Indonesia 2024 adalah Prabowo Subianto.  

['nama_mentri.pdf']
